In [1]:
import pandas as pd
import numpy as np
import matplotlib
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import TensorBoard
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train = pd.read_csv("train_ready_for_model.csv")
#train.drop(columns='DateTime',inplace=True)
train.head()

,product_A,product_B,product_C,product_D,product_E,product_F,product_G,product_H,product_I,product_J,...,Weekday_name_Sunday,Weekday_name_Thursday,Weekday_name_Tuesday,Weekday_name_Wednesday,session_id,user_id,is_click,pc_interest_count,day_view_count,day_interest_count
0,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,140690,858557,0.0,0.0,0.0,0.0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,333291,243253,0.0,0.0,0.0,0.0
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,129781,243253,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,464848,1097446,0.0,2.0,1.0,0.0
4,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,90569,663656,0.0,3.0,0.0,0.0


In [4]:
test = pd.read_csv("test_ready_for_model.csv")
#test.drop(columns='DateTime',inplace=True)
test.head()

,product_A,product_B,product_C,product_D,product_E,product_F,product_G,product_H,product_I,product_J,...,Weekday_name_Sunday,Weekday_name_Thursday,Weekday_name_Tuesday,Weekday_name_Wednesday,session_id,user_id,is_click,pc_interest_count,day_view_count,day_interest_count
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,411705,732573,NaN,1.0,0.0,0.0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,208263,172910,NaN,3.0,0.0,0.0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,239450,172910,NaN,3.0,0.0,0.0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,547761,557318,NaN,0.0,0.0,0.0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,574275,923896,NaN,0.0,0.0,0.0


In [5]:
result = test[["session_id"]]
result.head()

,session_id
0,411705
1,208263
2,239450
3,547761
4,574275


In [11]:
#Define target and ID columns:
target = 'is_click'
IDcol = ['session_id']
colorder=['session_id','is_click']
from sklearn import model_selection, metrics
from sklearn.metrics import classification_report
import time

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    t1=time.time()
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    kfold = model_selection.KFold(n_splits=10, random_state=7)

    #Perform cross-validation:
    cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='accuracy')
    fpr, tpr, tresholds = roc_curve(dtrain[target], dtrain[predictors])
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    print(classification_report(dtrain[target].values, dtrain_predictions))
    print ('AUC:', auc(fpr, tpr))
    print('ROC_AUC Score: ',roc_auc_score(dtrain[target], dtrain[predictors]))
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission=submission[colorder]
    submission.to_csv(filename, index=False)
    t2=time.time()
    t_linalg_lstsq = float(t2-t1)
    print("Time taken: {} seconds".format(t_linalg_lstsq))

In [12]:
from xgboost import XGBClassifier


alg1=XGBClassifier()

In [10]:
pandas_profiling.ProfileReport(train)

KeyboardInterrupt: 